In [1]:
import json
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
with open("Sarcasm_Headlines_Dataset.json", 'r') as f:
    datastore = json.load(f)

In [3]:
sentences = []
labels = []
urls = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [4]:
training_size = 20000
vocab_size = 10000
trunc_type='post'
padding_type='post'
embedding_dim = 16
max_length = 100
num_epochs = 30

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<oov>")
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length,
                                padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length,
                              padding=padding_type, truncating=trunc_type)

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
625/625 - 2s - loss: 0.6651 - accuracy: 0.5782 - val_loss: 0.5920 - val_accuracy: 0.6774
Epoch 2/30
625/625 - 1s - loss: 0.4453 - accuracy: 0.8237 - val_loss: 0.3891 - val_accuracy: 0.8401
Epoch 3/30
625/625 - 1s - loss: 0.3115 - accuracy: 0.8771 - val_loss: 0.3550 - val_accuracy: 0.8498
Epoch 4/30
625/625 - 1s - loss: 0.2551 - accuracy: 0.9010 - val_loss: 0.3529 - val_accuracy: 0.8465
Epoch 5/30
625/625 - 1s - loss: 0.2190 - accuracy: 0.9160 - val_loss: 0.3464 - val_accuracy: 0.8557
Epoch 6/30
625/625 - 1s - loss: 0.1918 - accuracy: 0.9270 - val_loss: 0.3579 - val_accuracy: 0.8512
Epoch 7/30
625/625 - 1s - loss: 0.1711 - accuracy: 0.9363 - val_loss: 0.3690 - val_accuracy: 0.8548
Epoch 8/30
625/625 - 1s - loss: 0.1531 - accuracy: 0.9443 - val_loss: 0.3831 - val_accuracy: 0.8544
Epoch 9/30
625/625 - 1s - loss: 0.1369 - accuracy: 0.9517 - val_loss: 0.4088 - val_accuracy: 0.8484
Epoch 10/30
625/625 - 1s - loss: 0.1244 - accuracy: 0.9572 - val_loss: 0.4241 - val_accuracy: 0.8496

In [10]:
sentence = [
    "Beaver builds statue at Beaver Dam for some random politician",
    "Well how about you do it instead"
]

sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length,
                      padding = padding_type,
                      truncating = trunc_type)

print(model.predict(padded))

[[9.9966860e-01]
 [1.7736429e-05]]
